#  Encaje de textos de **NHTSA Recalls** con `multilingual-e5-large-instruct`

## 1) Propósito

Implementar un **pipeline reproducible** para:

1. cargar y normalizar **chunks** de textos de *recalls* (preferentemente desde Parquet, con respaldo en JSONL);
2. computar **embeddings** con `intfloat/multilingual-e5-large-instruct` (normalizados para similitud coseno);
3. **persistir** los vectores y metadatos necesarios para auditoría, trazabilidad y consultas posteriores (p. ej., FAISS / Neo4j / RAG).

---

## 2) Entradas y salidas

### Entradas (en Google Drive)

* **Parquet (preferido):** `/content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet`
  Esquema esperado (detectado en ejecución):

  ```
  ['chunk_id', 'campaign_no', 'make', 'model', 'year', 'component',
   'chunk_idx', 'text_chunk']
  ```
* **JSONL (respaldo):** `/content/drive/MyDrive/NHTSA/processed/recalls_corpus.jsonl`
  Campos tolerados: `id`/`campaign_no`/`CAMPNO`, `text`/`text_full`/`passage`, y `metadata` opcional (`make`, `model`, `year`, `component`).

### Salidas (generadas)

En `BASE_OUT = /content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct`:

* **`recalls_embeddings.npy`** — matriz `float32` de shape `(n_chunks, dim)`.
* **`recalls_chunks_meta.parquet`** — metadatos por chunk (trazabilidad y auditoría).
* **`id_map.csv`** — mapeo simple `row -> (id, chunk_id)`.
* **`embedding_config.json`** — bitácora con hiperparámetros, tiempos y rutas efectivas.
* **`faiss_hnsw.index`** — *ruta reservada* (no se construye en este script).

---

## 3) Entorno y dependencias

* Instalación en Colab: `sentence-transformers`, `faiss-gpu` (con *fallback* a `faiss-cpu`).
* Librerías principales: `pandas`, `numpy`, `torch`, `tqdm`, `pathlib`, `hashlib`, `json`, `gc`, `time`.

Detección de dispositivo:

* `DEVICE = "cuda"` si hay GPU disponible; en caso contrario, `cpu`.

---

## 4) Estructura general del pipeline

### 4.1 Carga y normalización de datos

**Estrategia con prioridades:**

1. **Parquet ya chunked (preferido):** se lee el Parquet y se mapean columnas a un esquema canónico:

   * `id ← campaign_no`
   * `text ← text_chunk`
   * `chunk_id ← chunk_idx` (si no existe, se **deriva** por `cumcount()` agrupando por `id`)
   * `chunk_id_str ← chunk_id` original si existe, en otro caso `"{id}::ch{chunk_id}"`
   * Metadatos opcionales: `make`, `model`, `year`, `component`
   * Se **filtran** textos vacíos.
2. **Respaldo en JSONL:** si falla la ruta anterior, se reconstruyen chunks con política:

   * **`max_chars = 900`**
   * **`overlap = 120`**
   * Limpieza básica del texto (`" ".join(txt.split())`), y partición deslizante con solape.
   * Se generan `id`, `chunk_id`, `chunk_id_str`, `text` y metadatos si están en `metadata`.

**Salida normalizada de esta etapa:** `recalls_ch` con columnas mínimas

```
['id', 'chunk_id', 'chunk_id_str', 'text', 'make', 'model', 'year', 'component']
```

### 4.2 Encaje (embeddings)

* **Modelo:** `intfloat/multilingual-e5-large-instruct` (vía `SentenceTransformer`).
* **Normalización:** `normalize_embeddings = True` para usar **cosine similarity** directamente.
* **Batching:** `BATCH_SIZE = 64` (ajustable según VRAM).
* **Salida:** matriz `X` (`float32`) apilada con `np.vstack`, luego persistida en `recalls_embeddings.npy`.
* **Dimensión (`dim`):** detectada en tiempo de ejecución por el modelo.

### 4.3 Metadatos, auditoría y mapas

Se enriquecen metadatos por chunk con:

* `len_text` (longitud del texto),
* `model` (nombre del modelo),
* `device` (cuda/cpu),
* `dim` (dimensión del embedding),
* `ts` (timestamp UTC),
* `row_md5` (**hash MD5** por fila para trazabilidad), computado como:

  ```
  md5( f"{id}|{chunk_id}|{len_text}" )
  ```

Se guardan:

* **Metadatos parquet:** `recalls_chunks_meta.parquet`
* **Mapa indexado:** `id_map.csv` con columnas `row, id, chunk_id`
* **Bitácora:** `embedding_config.json` con:

  * `model`, `device`, `batch_size`, `normalize`,
  * `n_chunks`, `dim`,
  * `load_time_s`, `embed_time_s`,
  * rutas de `vectors_file`, `meta_file`, `map_file`.

> **Nota:** Se estima y reporta el tamaño de `X` en MB para anticipar costo de almacenamiento/memoria.

---

## 5) Políticas y criterios implementados

* **Normalización vectorial** obligatoria para comparación por coseno.
* **Chunking con solape** (900/120) para conservar contexto local y mitigar cortes semánticos duros.
* **Esquema canónico de columnas** para desacoplar el código de nombres específicos de origen.
* **Filtrado de vacíos** para estabilidad del encoder.
* **Tipado `float32`** para reducir huella (sin degradación relevante en búsqueda coseno).
* **Hash por fila** para trazabilidad y **detección de duplicados/cambios** (clave estable por `id|chunk_id|len_text`).

---

## 6) Resultados y artefactos (qué queda listo para usar)

1. **Embeddings listos para indexación**
   `recalls_embeddings.npy` + `recalls_chunks_meta.parquet` constituyen el par (vectores, metadatos) para:

   * indexación en **FAISS** (HNSW/IVF-Flat, etc.),
   * carga en **Neo4j** (como propiedad vectorial si se usa GDS/PGV con soporte),
   * **RAG** / agentes semánticos.

2. **Auditoría y reproducibilidad**
   `embedding_config.json` fija parámetros, dimensiones y tiempos (carga/encaje).
   `row_md5` en metadatos permite validar integridad por fila.

3. **Navegabilidad/joins**
   `id_map.csv` facilita cruces rápidos `(row -> id, chunk_id)` en exploraciones y *debug*.

> **Pendiente intencional:** el **índice FAISS** no se construye en este cuaderno (solo se reserva la ruta `faiss_hnsw.index`). Esto permite separar “cálculo de embeddings” de “estrategias de indexación” (HNSW vs IVF, métricas, parámetros K/efSearch/efConstruction, etc.).

---

## 7) Verificación mínima sugerida (post-ejecución)

* **Conteo y dimensión**

  * `n_chunks` en `embedding_config.json` coincide con `recalls_embeddings.npy.shape[0]`.
  * `dim` de `embedding_config.json` coincide con `recalls_embeddings.npy.shape[1]` y con la columna `dim` de `recalls_chunks_meta.parquet`.

* **Trazabilidad**

  * Muestras aleatorias: `id`, `chunk_id`, `chunk_id_str` y `len_text` deben corresponder con lo esperado del origen Parquet/JSONL.
  * `row_md5` estable ante recarga si no cambian `id`, `chunk_id` ni el contenido del texto.

* **Calidad básica del corpus**

  * Distribución de `len_text` sin picos anómalos (p. ej., excesivos vacíos, *outliers* sin chunking).
  * Metadatos (`make`, `model`, `year`, `component`) presentes cuando existen en la fuente.

---

## 8) Consideraciones de rendimiento

* **Batching** (`BATCH_SIZE = 64`) dimensionado para Colab “estándar”; en GPU con más VRAM puede aumentarse para reducir tiempo total.
* **Tiempo de carga/encaje** reportado como `load_time_s` y `embed_time_s` en `embedding_config.json`.
* **Ahorro de memoria** al forzar `float32` y *streaming* por lotes sin materializar listas intermedias enormes.

---

## 9) Limitaciones conocidas

* **FAISS no construido** en este notebook: se pospone a una etapa posterior con elección de índice (HNSW/IVF) y *tuning* de parámetros.
* **Suposiciones de esquema**: si el Parquet no presenta columnas clave (p. ej. `campaign_no`, `text_chunk`), se recurre al **fallback JSONL** con re-chunking, lo que puede cambiar el particionado respecto a otras fuentes.
* **Normalización** fija a `True` (adecuada para coseno). Si se usaran otras métricas/índices, evaluar implicaciones.

---

## 10) Próximos pasos recomendados

1. **Construcción del índice FAISS**

   * HNSW (cosine) con `M`, `efConstruction`, `efSearch` adecuados al tamaño final.
   * Persistir en `faiss_hnsw.index` y validar *recall@k*.

2. **Validación semántica**

   * *Smoke tests*: búsquedas por `campaign_no`, `component` y descripciones conocidas para confirmar coherencia semántica de los vecinos.

3. **Integración con motores de consulta**

   * **Neo4j**: almacenar `id`, `chunk_id_str`, metadatos y vector (o id FAISS) para consultas híbridas (filtro estructural + similitud).
   * **RAG/Agentes**: conectar índice + metadatos a un *retriever* (coseno) con formateo de contexto por `chunk_id_str`.

4. **Monitoreo y control de versiones**

   * Versionar `embedding_config.json` y conservar `row_md5` como *fingerprint* por chunk.
   * Si cambia la política de chunking o el modelo, regenerar y dejar ambas versiones disponibles.

---

## 11) Resumen ejecutivo

* Se ejecutó un pipeline **determinista** para encajar textos de *recalls* en chunks con **solape 900/120**, priorizando Parquet normalizado y usando JSONL como respaldo.
* Se calcularon **embeddings normalizados** con `intfloat/multilingual-e5-large-instruct` en lotes, y se **persistieron** vectores y metadatos exhaustivos (incluido **hash por fila**, **timestamp**, **dimensión**, **dispositivo**).
* Se dejó lista la **base de artefactos** para indexación (FAISS), consultas semánticas y auditoría.
* La construcción del **índice FAISS** quedó **intencionadamente fuera** de este notebook y se recomienda abordarla como siguiente etapa.


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CE L D A  1  ·  Setup básico (Drive, librerías, paths)
# ─────────────────────────────────────────────────────────────────────────────
!pip -q install -U sentence-transformers faiss-gpu || pip -q install -U faiss-cpu

from pathlib import Path
import os, json, gc, math, time, hashlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch

from sentence_transformers import SentenceTransformer

# Monta Drive si estás en Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

# Rutas base (ajusta si lo necesitas)
BASE_IN   = Path("/content/drive/MyDrive/NHTSA/processed")
BASE_OUT  = Path("/content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct")
BASE_OUT.mkdir(parents=True, exist_ok=True)

# Fuentes posibles
PARQUET_CHUNKS = BASE_IN / "recalls_chunks_fixed.parquet"   # preferido (ya chunked)
JSONL_CORPUS   = BASE_IN / "recalls_corpus.jsonl"           # fallback si no existe el Parquet

# Archivo(s) de salida
META_OUT   = BASE_OUT / "recalls_chunks_meta.parquet"       # metadatos (por chunk)
VEC_NPY    = BASE_OUT / "recalls_embeddings.npy"            # matriz de embeddings float32 (n_chunks, dim)
FAISS_IDX  = BASE_OUT / "faiss_hnsw.index"                  # índice HNSW opcional
MAP_CSV    = BASE_OUT / "id_map.csv"                        # mapeo fila -> (id, chunk_id)
CONF_JSON  = BASE_OUT / "embedding_config.json"             # bitácora de configuración

# Parámetros del modelo / cómputo
MODEL_NAME   = "intfloat/multilingual-e5-large-instruct"
BATCH_SIZE   = 64                   # sube/baja según VRAM
NORMALIZE    = True                 # recomendado para similitud coseno
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {DEVICE}")
print(f"Salida: {BASE_OUT}")

# Limpieza previa si quieres forzar una corrida desde cero
# for p in [META_OUT, VEC_NPY, FAISS_IDX, MAP_CSV]:
#     if p.exists(): p.unlink()


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cuda
Salida: /content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct


In [ ]:
# Inspector: mira qué columnas trae tu Parquet y 5 filas de ejemplo
from pathlib import Path
import pandas as pd

PARQUET_CHUNKS = Path("/content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet")
JSONL_CORPUS   = Path("/content/drive/MyDrive/NHTSA/processed/recalls_corpus.jsonl")

if PARQUET_CHUNKS.exists():
    df_ins = pd.read_parquet(PARQUET_CHUNKS)
    print("Archivo encontrado:", PARQUET_CHUNKS, "| Filas x Cols:", df_ins.shape)
    print("Columnas:", list(df_ins.columns))
    display(df_ins.head(5))
else:
    print("No existe el Parquet de chunks:", PARQUET_CHUNKS)

print("\nExiste JSONL de respaldo?:", JSONL_CORPUS.exists(), JSONL_CORPUS)


Archivo encontrado: /content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet | Filas x Cols: (12871, 8)
Columnas: ['chunk_id', 'campaign_no', 'make', 'model', 'year', 'component', 'chunk_idx', 'text_chunk']


,chunk_id,campaign_no,make,model,year,component,chunk_idx,text_chunk
0,10E002000::ch0,10E002000,TOYOTA,CELICA,1986,SUSPENSION,0,RIDE CONTROL IS RECALLING CERTAIN FRONT STRUT ...
1,10E013000::ch0,10E013000,DODGE,RAM 2500,2007,STEERING:GEAR BOX:SHAFT PITMAN,0,FABTECH IS RECALLING CERTAIN AFTERMARKET REPLA...
2,10E019000::ch0,10E019000,NISSAN,TITAN,2010,SUSPENSION:FRONT:CONTROL ARM:LOWER ARM,0,NISSAN IS RECALLING CERTAIN FRONT AND REAR LOW...
3,10E021000::ch0,10E021000,BMW,740I,1996,STEERING:LINKAGES:DRAG:LINK:CONNECTION,0,FCP GROTON IS RECALLING CERTAIN STEERING CENTE...
4,10E034000::ch0,10E034000,DODGE,CARAVAN,2007,EQUIPMENT,0,CURT MFG IS RECALLING CERTAIN CLASS 3 RECEIVER...



Existe JSONL de respaldo?: True /content/drive/MyDrive/NHTSA/processed/recalls_corpus.jsonl


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CE L D A  2B  ·  Carga robusta (ajustada a tu esquema detectado)
# ─────────────────────────────────────────────────────────────────────────────
from pathlib import Path
import pandas as pd, json

PARQUET_CHUNKS = Path("/content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet")
JSONL_CORPUS   = Path("/content/drive/MyDrive/NHTSA/processed/recalls_corpus.jsonl")

def basic_chunk_text(txt, max_chars=900, overlap=120):
    if not isinstance(txt, str): return []
    s = " ".join(txt.split())
    if len(s) <= max_chars:
        return [s]
    out, i = [], 0
    step = max_chars - overlap
    while i < len(s):
        out.append(s[i:i+max_chars])
        i += step
    return out

def load_chunks_df():
    # 1) Preferir tu parquet ya chunked
    if PARQUET_CHUNKS.exists() and PARQUET_CHUNKS.stat().st_size > 0:
        df = pd.read_parquet(PARQUET_CHUNKS)
        print("[INFO] Parquet encontrado:", PARQUET_CHUNKS, "| shape:", df.shape)
        print("[INFO] Columnas:", list(df.columns))

        # Caso exacto que reportaste:
        # ['chunk_id','campaign_no','make','model','year','component','chunk_idx','text_chunk']
        # Mapeo a columnas canónicas:
        col_id        = "campaign_no" if "campaign_no" in df.columns else None
        col_text      = "text_chunk"  if "text_chunk"  in df.columns else None
        col_chunk_num = "chunk_idx"   if "chunk_idx"   in df.columns else None
        col_chunk_str = "chunk_id"    if "chunk_id"    in df.columns else None  # ej. "10E002000::ch0"

        # Validaciones mínimas
        assert col_id is not None,   "No se encontró columna 'campaign_no' para ID."
        assert col_text is not None, "No se encontró columna 'text_chunk' para el texto."
        # Si no hubiera chunk_idx, se generará abajo

        out = pd.DataFrame({
            "id":   df[col_id].astype(str),
            "text": df[col_text].astype(str).str.strip()
        })

        # chunk_id numérico
        if col_chunk_num in df.columns:
            out["chunk_id"] = pd.to_numeric(df[col_chunk_num], errors="coerce").fillna(0).astype(int)
        else:
            # si falta, derivarlo por grupo de id
            out = out.sort_values(["id"]).reset_index(drop=True)
            out["chunk_id"] = out.groupby("id").cumcount()

        # Mantener identificador string original si existe (útil para trazabilidad)
        if col_chunk_str in df.columns:
            out["chunk_id_str"] = df[col_chunk_str].astype(str)
        else:
            out["chunk_id_str"] = out["id"].astype(str) + "::ch" + out["chunk_id"].astype(str)

        # Metadatos útiles si están
        for meta in ["make","model","year","component"]:
            if meta in df.columns:
                out[meta] = df[meta]

        # Filtrar vacíos
        out = out[out["text"].str.len() > 0].reset_index(drop=True)
        print("[INFO] Chunks cargados y normalizados:", out.shape)
        return out

    # 2) Fallback: JSONL → re-chunk
    assert JSONL_CORPUS.exists(), f"No se encontró {PARQUET_CHUNKS} ni {JSONL_CORPUS}"
    print("[WARN] No hay Parquet válido; se construirán chunks desde JSONL:", JSONL_CORPUS)

    rows = []
    with open(JSONL_CORPUS, "r", encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            _id  = obj.get("id") or obj.get("campaign_no") or obj.get("CAMPNO")
            txt  = obj.get("text") or obj.get("text_full") or obj.get("passage") or ""
            md   = obj.get("metadata") or {}
            chunks = basic_chunk_text(txt, max_chars=900, overlap=120)
            for j, ch in enumerate(chunks):
                rows.append({
                    "id": _id,
                    "chunk_id": j,
                    "chunk_id_str": f"{_id}::ch{j}",
                    "text": ch.strip(),
                    "make": md.get("make"),
                    "model": md.get("model"),
                    "year": md.get("year"),
                    "component": md.get("component"),
                })
    out = pd.DataFrame(rows)
    out = out[out["text"].str.len() > 0].reset_index(drop=True)
    print("[INFO] Chunks construidos desde JSONL:", out.shape)
    return out

recalls_ch = load_chunks_df().copy()
print("\nVista previa normalizada:")
display(recalls_ch.head(5)[["id","chunk_id","chunk_id_str","text","make","model","year","component"]])


[INFO] Parquet encontrado: /content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet | shape: (12871, 8)
[INFO] Columnas: ['chunk_id', 'campaign_no', 'make', 'model', 'year', 'component', 'chunk_idx', 'text_chunk']
[INFO] Chunks cargados y normalizados: (12871, 8)

Vista previa normalizada:


,id,chunk_id,chunk_id_str,text,make,model,year,component
0,10E002000,0,10E002000::ch0,RIDE CONTROL IS RECALLING CERTAIN FRONT STRUT ...,TOYOTA,CELICA,1986,SUSPENSION
1,10E013000,0,10E013000::ch0,FABTECH IS RECALLING CERTAIN AFTERMARKET REPLA...,DODGE,RAM 2500,2007,STEERING:GEAR BOX:SHAFT PITMAN
2,10E019000,0,10E019000::ch0,NISSAN IS RECALLING CERTAIN FRONT AND REAR LOW...,NISSAN,TITAN,2010,SUSPENSION:FRONT:CONTROL ARM:LOWER ARM
3,10E021000,0,10E021000::ch0,FCP GROTON IS RECALLING CERTAIN STEERING CENTE...,BMW,740I,1996,STEERING:LINKAGES:DRAG:LINK:CONNECTION
4,10E034000,0,10E034000::ch0,CURT MFG IS RECALLING CERTAIN CLASS 3 RECEIVER...,DODGE,CARAVAN,2007,EQUIPMENT


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CE L D A  3  ·  Encaje con intfloat/multilingual-e5-large-instruct
# ─────────────────────────────────────────────────────────────────────────────

# Carga del modelo
t0 = time.time()
model = SentenceTransformer(MODEL_NAME, device=DEVICE)
load_time = time.time() - t0
print(f"Modelo cargado en {load_time:.2f}s")

texts = recalls_ch["text"].tolist()
n = len(texts)
print(f"Total de chunks a encajar: {n}")

# Embedding (stream/batches) con normalización opcional
embeds = []
t1 = time.time()
for i in tqdm(range(0, n, BATCH_SIZE), desc="Embedding"):
    batch = texts[i:i+BATCH_SIZE]
    vec = model.encode(
        batch,
        batch_size=len(batch),
        show_progress_bar=False,
        convert_to_numpy=True,
        normalize_embeddings=NORMALIZE
    )
    # Asegurar float32 para ahorrar espacio
    vec = vec.astype("float32", copy=False)
    embeds.append(vec)
embed_time = time.time() - t1

X = np.vstack(embeds)
dim = X.shape[1]
np.save(VEC_NPY, X)

print(f"Embeddings: shape={X.shape}, dtype={X.dtype}, guardados en {VEC_NPY}")
print(f"Tiempo de embedding: {embed_time:.2f}s  |  Dimensión: {dim}")

# Metadatos + mapa
recalls_ch = recalls_ch.copy()
recalls_ch["len_text"] = recalls_ch["text"].str.len()
recalls_ch["model"]    = MODEL_NAME
recalls_ch["device"]   = DEVICE
recalls_ch["dim"]      = dim
recalls_ch["ts"]       = pd.Timestamp.utcnow()

# Hash de fila útil para auditoría
def row_hash(r):
    h = hashlib.md5()
    key = f"{r['id']}|{r['chunk_id']}|{r['len_text']}"
    h.update(key.encode("utf-8"))
    return h.hexdigest()

recalls_ch["row_md5"] = recalls_ch.apply(row_hash, axis=1)

# Guardar metadatos / mapa
recalls_ch.to_parquet(META_OUT, index=False)
pd.DataFrame({"row": np.arange(n), "id": recalls_ch["id"], "chunk_id": recalls_ch["chunk_id"]}).to_csv(MAP_CSV, index=False)

# Bitácora de configuración
conf = {
    "model": MODEL_NAME,
    "device": DEVICE,
    "batch_size": BATCH_SIZE,
    "normalize": NORMALIZE,
    "n_chunks": int(n),
    "dim": int(dim),
    "load_time_s": round(load_time, 3),
    "embed_time_s": round(embed_time, 3),
    "vectors_file": str(VEC_NPY),
    "meta_file": str(META_OUT),
    "map_file": str(MAP_CSV)
}
with open(CONF_JSON, "w") as f:
    json.dump(conf, f, indent=2)

# Resumen de almacenamiento
mb_vectors = (X.nbytes)/(1024**2)
print(f"≈ Tamaño de matriz de embeddings: {mb_vectors:.2f} MB")
print("Meta parquet:", META_OUT)
print("Map CSV     :", MAP_CSV)


Modelo cargado en 6.76s
Total de chunks a encajar: 12871


Embedding:   0%|          | 0/202 [00:00<?, ?it/s]

Embeddings: shape=(12871, 1024), dtype=float32, guardados en /content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct/recalls_embeddings.npy
Tiempo de embedding: 850.60s  |  Dimensión: 1024
≈ Tamaño de matriz de embeddings: 50.28 MB
Meta parquet: /content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct/recalls_chunks_meta.parquet
Map CSV     : /content/drive/MyDrive/NHTSA/embeddings/recalls_e5_mlg_instruct/id_map.csv
